In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
TA_REVIEW_DATA_PATH = os.path.join("..","..","data","TA_User_Reviws_Korea_all.json")
df = pd.read_json(TA_REVIEW_DATA_PATH )

In [5]:
df.head(5)

,id,language,reviewStatus,title,helpfulVotes,createdDate,publishedDate,rating,text,photoIds,...,location.thumbnail,route,location.route,userProfile.route,location.reviewSummary,location.parent,userProfile.hometown.location.parent,userProfile.contributionCounts,location,userProfile.avatar
0,748454819,en,PUBLISHED,Great Hotel,0,2020-03-01,2020-03-01,5,I really enjoyed our stay here in Gracery Hote...,[453807126],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,744980685,en,PUBLISHED,Vibrant and Trendy,0,2020-02-14,2020-02-13,5,This is a place for you to laidback and have s...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,744978150,en,PUBLISHED,So Yummy!,0,2020-02-14,2020-02-13,4,The chicken was so delicious. its crisp and cr...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,744977340,en,PUBLISHED,Super Luv,0,2020-02-14,2020-02-13,5,"First, the location. Its located just right be...",[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,744973034,en,PUBLISHED,Chaotic Street Food,0,2020-02-14,2020-02-13,4,This is the place where you can shop any cosme...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['location.placeType'].value_counts()

ACCOMMODATION    14421
EATERY           12118
ATTRACTION       10540
ACTIVITY           117
CITY                 1
Name: location.placeType, dtype: int64

In [7]:
df[df['location.placeType'] == "EATERY"]['location.additionalNames.long'].value_counts()

Myeongdong Kyoja Main, South Korea                        166
Tosokchon Samgyetang, South Korea                         126
Yoogane Myeongdong 1st Store, South Korea                  37
Isaac Toast Myeongdong, South Korea                        37
Sinseon Seolleongtang Myeongdong, South Korea              31
                                                         ... 
Guryongpo Gyejeol Hoejib, South Korea                       1
Seongsu Kalguksu, South Korea                               1
Starbucks Jinju Gyeongsang National Univ., South Korea      1
Yugane Dakgalbi Cheonho, South Korea                        1
Hyeongje Yukheo, South Korea                                1
Name: location.additionalNames.long, Length: 7066, dtype: int64